## This is an individual assignment. Do not work in groups, and do not consult each other when doing this midterm.

# Task 1 - Getting tweets (3 points)

Pick either two Twitter usernames. These usernames should be "oppositional" in nature. I'm leaving the exact meaning of the term "oppositional" intentional vague; the two usernames you pick should be in the same domain (politics, products, pop culture, etc.) but should also be strong contrasts or even rivals. For example:  @Apple vs. @SamsungMobile, @SenMajLdr vs. @SenSchumer, @CocaCola vs. @pepsi, @OfficialKanye vs. @taylorswift13. Basically, if the two handles/hashtags you pick can sensibly fit into the phrase "A vs. B," then they're oppositional.

In the code block below, use the twitter API to get **the text** of 500 tweets representing the two usernames you pick - specifically, the 500 most recent tweets of the users. Please pick users that tweet primarily in English. This "English" requirement is simply so the two instructors can go through the information. This means you'll have a total of 1000 tweets.

**Do NOT pick any of the examples I gave above above as your two oppositional usernames. Think up an original pair. Note that this task requires you to come up with two "oppositional" entities as well as to get those entities most recent tweets. Keep in mind that this is an individual assignment. Any two students who happen to pick the same two oppositional entities may be double checked for cheating, therefore, the more "original" your oppositional pair, the better. Any two students who have exactly the same set of tweets will draw a high amount of suspicion, since it is highly unlikely students will a) pick the same entities and b) get their tweets at exactly the same time.**

In [1]:
import got3 as gt

In [2]:
# Extracting tweets from Playstation Official Twitter Account
tC1 = gt.manager.TweetCriteria();
tC1.setSince("2017-9-23")
tC1.setUntil("2017-12-31")
tC1.setUsername('playstation')
tC1.setLang("en")
tC1.setMaxTweets(500);

In [3]:
from time import time
t0 =  time()
ps_tweets = gt.manager.TweetManager.getTweets(tC1)
print("Time Taken is %0.3f seconds" % (time() - t0))
len(ps_tweets)

Time Taken is 38.429 seconds


500

In [4]:
tweet = ps_tweets[19]
print(tweet.text)

The season of savings continues with Week 3 of PS Store’s Holiday Sale! Save on games like Destiny 2, Shadow of War, Crash Bandicoot N. Sane Trilogy, and more: http://play.st/2Acws9d pic.twitter.com/CwRaz6KBPQ


In [5]:
# Extracting tweets from Xbox Official Twitter Account
tC2 = gt.manager.TweetCriteria();
tC2.setSince("2017-9-23")
tC2.setUntil("2017-12-31")
tC2.setUsername("xbox")
tC2.setLang("en")
tC2.setMaxTweets(500);

In [6]:
t0 =  time()
xbox_tweets = gt.manager.TweetManager.getTweets(tC2)
print("Time Taken is %0.3f seconds" % (time() - t0))
len(xbox_tweets)

Time Taken is 42.942 seconds


500

In [7]:
tweet = xbox_tweets[27]
print(tweet.text)

We bet you did great!


## Task 1 Rubric

- code to obtain Tweets runs without errors (1 point)
- code correctly written to obtain the latest 500 tweets from a pair of usernames (1 point)
- student REMOVES their API key and secret before submitting the midterm!!! (1 point)

# Task 2 - Saving your data to an external file (3 points)

In the code block below, consolidate your tweets into a single variable. This variable should have two "columns," one for the text of the tweets, the other a binary indicator of the source (e.g. 0 for the first source, 1 for the second). You can use a list of tuples or A Pandas dataframe, and save as either a pickle (.pkl) or a comma-seperated values file (.csv).

In [8]:
import pandas as pd
ps_ds = [[x.text, "0"] 
            for x in ps_tweets]
ps_df = pd.DataFrame(data=ps_ds, columns=[ "Text", "Label"])

In [9]:
ps_df.head()

,Text,Label
0,Travel back to Middle-earth and wage all-out w...,0
1,Game on.https://twitter.com/espn/status/947212...,0
2,Less than two hours til the PlayStation @ Fies...,0
3,On the latest episode of PlayStation Undergrou...,0
4,Look back at five years of http://PS.Blog 's G...,0


In [10]:
xbox_ds = [[x.text, "1"] 
            for x in xbox_tweets]
xbox_df = pd.DataFrame(data = xbox_ds, columns=[ "Text", "Label"])
xbox_df.head()

,Text,Label
0,"In 2017, we launched the world's most powerful...",1
1,100+ games or multiplayer with your buddies? W...,1
2,Ready to experience the world’s most powerful ...,1
3,Fewer side effects. More games. Get started wi...,1
4,"Optional: shirt, shoes, pants. # PUBG [T] http...",1


In [11]:
data = pd.concat([ps_df,xbox_df], ignore_index= True)
data.to_csv('SMM_Midterm_Data' + '.csv',index= False)

## Task 2 Rubric

- code to save data to external file runs without errors (1 point)
- saved data formatted correctly into "2 columns" (1 point)
- external file submitted with midterm (1 point)

# Task 3 - Preparing data for an sklearn binary classifier (3 points)

In the code block below, create two variables, X, and y. The y variable should be simple - it is simply the "second column" of the data you made in task 2, a binary indicator of source, with 0 representing one source and 1 representing the other.

For the X variable, choose either `TfidifVectorizer` or `CountVectorizer` from `sklearn.feature_representation.text` to turn the raw text (column 1 from task 2) into a "bag-of-words" representation. When instantiating your vectorizer, set the argument `lowercase=True`, to ensure that all words are lowercased, and `stopwords="english"`, to remove English stop words. 

Additional, when instantiating the vectorizer, pass the `max_df=???` and `min_df=???` arguments. These arguments can either take a *float between 0.0 and 1.0 or an integer*. The df stands for "document frequency." These arguments tell the vectorizer to remove words that occur *over* (max_df) and *under* (min_df) a certain amount of documents. This will remove frequent words - which show up all the time and therefore are not informative - and infrequent words, which are so rare as to just be noise. If you pass these arguments a float, that float represents the proportion of documents (e.g. `max_df=0.9` means, remove all words that show up in more than 90% of the tweets) and if you pass these arguments an integer, that integer represents the number of documents (e.g. `min_df=5`, remove all words that show up in less than 5 document). 

By the end of this task, you should have a variable X, of dimensionality $n \times d$ where $n = 1000$ and $d$ is the number of words left after the vectorizer considers df, and you should have variable y, which is a vector of length 1000, with 1s or 0s representing tweet source.

In [12]:
# Pre-processing for a Binary Classifier
text_data = data
print(text_data.shape)
text_data.groupby('Label').count()

(1000, 2)


,Text
Label,
0,500
1,500


In [13]:
X = text_data.Text
y = text_data.Label

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(lowercase = True , stop_words = "english", min_df = 1, max_df = 0.9)
vec.fit(X)
train_matrix = vec.transform(X)
print(train_matrix.shape)
train_df = pd.DataFrame(train_matrix.toarray(), columns=vec.get_feature_names())

(1000, 3095)


## Task 3 Rubric

- Code runs without errors (1 point)
- Written code correctly achieves objective of creating X, y variables for classifier (1 point)
- All required arguments to vectorizer included (1 point)


# Task 4 - Training a Logistic Regression classifier (3 points)

Instantiate an sklearn Logistic Regression binary classifier ([sklearn documentation here](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)). 

Then, use `cross_val_score` from `sklearn.model_selection` ([sklearn documentation here](http://scikit-learn.org/stable/modules/cross_validation.html#computing-cross-validated-metrics))to perform 5-fold cross validation. The inputs to `cross_val_score` will be your instantiated Logistic Regression classifier, X, y, and a named argument `cv=5` to indicate the number of folds. The output will be a list of 5 numbers - the accuracy from each fold.

Print the average of those 5 numbers. This will be the mean 5-fold cross validation accuracy of your classifier.

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df, y, test_size = 0.20,random_state = 2)

In [16]:
# Performing Logistic Regression without Cross-Validation
from sklearn.linear_model import LogisticRegression
logreg_clf = LogisticRegression()
t0 = time()
logreg_clf.fit(X_train,y_train);
print("Completed in %0.3fs" % (time() - t0))

Completed in 0.040s


In [17]:
y_pred = logreg_clf.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred) #Checking for the Accuracy

0.98499999999999999

In [18]:
# Performing Cross-Validation
from sklearn.model_selection import cross_val_score
cv_scores_lr = cross_val_score(logreg_clf, train_df, y, cv=5, scoring="accuracy")
print(cv_scores_lr)
print("Accuracy: %0.2f (+/- %0.2f)" % (cv_scores_lr.mean(), cv_scores_lr.std() * 2)) # 95% Confidence Interval

[ 0.985  0.97   0.98   0.99   0.975]
Accuracy: 0.98 (+/- 0.01)


In [19]:
# Multinomial Naive-Bayes without Cross-Validation
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB();
t0 = time()
mnb.fit(X_train,y_train);
print("Completed in %0.3fs" % (time() - t0))

Completed in 0.037s


In [20]:
y_pred = mnb.predict(X_test)
accuracy_score(y_test,y_pred) #Checking for the Accuracy

0.96999999999999997

In [21]:
# Performing Cross-Validation
cv_scores_mnb = cross_val_score(mnb, train_df, y, cv=5, scoring="accuracy")
print(cv_scores_mnb)
print("Accuracy: %0.2f (+/- %0.2f)" % (cv_scores_mnb.mean(), cv_scores_mnb.std() * 2)) # 95% Confidence Interval

[ 0.96   0.905  0.95   0.955  0.965]
Accuracy: 0.95 (+/- 0.04)


## Task 4 Rubric

- Code runs without errors (1 point)
- Code successfully creates a Logistic Regression classifier and runs cross validation (1 point)
- Code prints mean cross validation accuracy (1 point)

# Task 5 - Discussion

Answer the following questions.

1 Since you pulled equal amounts of tweets from each source, the baseline accuracy is 50%. This is the accuracy we would expect from a classifier that guessed 0 or 1 randomly, or a classifier that simply guessed all 0s or all 1s. Your classifier either did well or did poorly. In either case, think about the *actual content* of the sources you picked and in the text block below, informally share your thoughts on why your classifier did poorly/well. (1 point)

The classifier performed excpetionally well on the data it managed to achieve an accuracy of 98 percent on average, the same can be seen above when the average score with a 95 CI has been displayed after cross validation. Since our sample size is only 1000 tweets, it would be interesting to see if similar results can be obtained when there are millions of tweets.

2 What could you have done differently when preprocessing your data (task 3) to try and improve your classifier's accuracy? (1 point)

During preprocessing we could use a pipeline of a both a CountVectorizer and a TfidfTransformer using the Pipeline model in sklearn, also stopwords
might or might not have an effect on our classifier we have removed these and trained our model. Other NLP techniques such as stemming could be used.

3 What parameters could you have adjusted in the Logistic Regression classifier in Task 4 to "tune" it and get better performance? What other binary classifiers could you have used, and what "tune-able" parameters do those classifiers have? (1 point)

The parameter 'penalty' is the main tunable option, other parametrs include max_iter, n_jobs. The code for using a Multinomial Naive Bayes for our data has been included. MNB performs generally well on text data. The parameter'alpha' is tunable for smoothing.

# Bonus (3 points)

This is an opportunity to gain an additional 3 points above the 15 allocated for the midterm. This will require you to do something that wasn't covered directly in the lectures, but can somewhat easily be learned by going through the sklearn documentation.

Note 1: When a count vectorizer or TF-IDF vectorizer is instantiated and used to transform your raw text data, it builds a dictionary that indicates which word is assigned to which index. Remember that it produces an $n \times d$ matrix, where $n$ is the number of samples and $d$ is the number of words. If you want to know the index of a word (that is, which column in $d$ corresponds to that word), you can consult this dictionary. Suppose you named your vectorizer `vec`. To access this dictionary, use `vec.vocabulary_`. If you want to know the index of the word `banana`, access `vec.vocabulary_['banana']`. 

Note 2: When you instantiate and train a logistic regression, it saves a set of *coefficients* indicating the "weight" of that word in terms of predicting the outcome variable. Suppose you named your classifier `lr`. You can access these coefficents at `lr.coef_[1]`. (the `[1]` is there because `lr.coef_[0]` is where the intercept of the model is stored). This means that `lr.coef_[1][0]` is the weight of the 0th feature, `lr.coef_[1][1]` is the weight of the 1st feature, and so on.

You can therefore *iterate* through `vec.vocabulary_.items()`, and for each word (key) get its index (value) and then find the coefficient weight of that word in the model `lr.coef_[1][index]`. 

In the code block (or blocks, if you want to make more than one to organize your code better) below:

1. Instantiate a *new* instance of a Logistic Regression classifier, `fit` that classifier on X and y. (1 point)
2. Use the notes above to make a list of tuples, where the first value in each tuple is a *word in the vocabulary* and the second value is the *coefficient weight assigned to that word in the trained Logistic Regression classifier*. Sort that list of tuples by the second value (the weight) ([Here's how you can do that](https://stackoverflow.com/questions/10695139/sort-a-list-of-tuples-by-2nd-item-integer-value)). (1 point)
3. Print the 10 words with the highest weights and the 10 words with the lowest weights. In a few sentences discuss whether these words help you understand why the model performed well/poorly. (1 point).

In [22]:
# Performing Logistic Regression without Cross-Validation
lr = LogisticRegression()
t0 = time()
lr.fit(train_df,y); # Not using train_test split fitting the entire data
print("Completed in %0.3fs" % (time() - t0))

Completed in 0.049s


In [23]:
words = []
n = 0
for i in vec.vocabulary_.items():
    n = i[1]
    word = i[0]
    weight = lr.coef_[0][n]
    w = (word,weight)
    words.append(w)

In [24]:
final_words = sorted(words, key = lambda x: x[1])

In [25]:
# Top 10 words with least weights
for i in range(10):
    print(final_words[i])

('com', -2.3461623472930704)
('psx', -2.0705321430657033)
('twitter', -1.9745080428172177)
('st', -1.7910023430212125)
('ps4', -0.89111074867263629)
('https', -0.84475849188437102)
('big', -0.83925495316108845)
('play', -0.7835420942260487)
('pic', -0.77559829120235146)
('playstation', -0.75824900315047172)


In [26]:
# Top 10 words with highest weights
reverse = final_words.reverse()
for i in range(10):
    print(final_words[i])

('ly', 1.7913701022367441)
('bit', 1.7913701022367441)
('xbox', 0.97383682039297148)
('xboxpic', 0.84820165183528129)
('great', 0.62183110674488884)
('xboxonepic', 0.60067023527288288)
('console', 0.58769651185098193)
('fun', 0.5610477152353881)
('love', 0.51957226998152828)
('good', 0.47978569958074641)


Part 3: In a few sentences discuss whether these words help you understand why the model performed well/poorly (1 point)

Most of the words with high weights are very commonly associated with username/criteria we have used for mining tweets, so the words though generic at times must have been helpful in classifying.